<a href="https://colab.research.google.com/github/tanvu10/Hidden_Markov_Model/blob/master/Base_Hidden_Markov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [ ]:
class HiddenMarkov():
    def __init__(self, A, B, init_state_vec, states = None, obs = None):
        'A: transition matrix: NxN'
        'B: emission matrix: NxM'
        'init_state_vec: Nx1'
        'N: state num'
        'M: observation num'
        self.A = A
        self.B = B
        self.pi = init_state_vec
        self.state_num = A.shape[0]
        self.obs_num = B.shape[1]
        self.states = states  
        self.obs = obs
        self.state_dic_map()
        self.observation_dic_map()
        self.stage_num = None


    def state_dic_map(self):
        self.state_dic ={}
        for index, states in enumerate(self.states):
            self.state_map[index] = states
    
    def observation_dic_map(self):
        self.observation_dic ={}
        for index, observation in enumerate(self.obs):
            #map input to index of observation
            self.observation_dic[observation] = index
    
    def alpha(self, observation):
        'using forward algorithm'
        alpha = np.zeros((self.stage_num, self.state_num)) #matrix 
        c = np.zeros((self.stage_num, 1)) #column vec


        first_obs = self.observation_dic[observation[0]]
        #alpha0: for t = 0 (column_vec)
        alpha[0,:] = np.multiply(self.B[:,first_obs], self.pi).T 
        #C_scale 
        c[0,0] = 1/np.sum(alpha[0,:])
        alpha[0,:] = alpha[0,:]*c[0,0]


        #forward alpha:
        for i in range(1,self.stage_num):
            #transition_vec to stage i
            alpha[i,:] = (self.A.T.dot(alpha[i-1, :]))  *   self.B[:, self.observation_dic[obervation[i]]].T   #row vector
            c[i,0] = 1/np.sum(alpha[i,:])
            alpha[i,:] = alpha[i,:]*c[i,0]
        return alpha, c

    def beta(self,observation, c):
        beta = np.zeros((self.stage_num, self.state_num))
        
        #beta at time T:
        beta[self.stage_num-1,:] = 1*c[stage_num-1,:]

        #backward beta
        for i in reversed(range(stage_num-1)):
            beta[i,:] = np.dot(A, (B[:, self.observation_dic[observation[i+1]]] *  beta[i+1,:].T  ).T   ).T
            beta[i,:] = beta[i,:]*c[i,:]

        return beta

    def gamma(self, observation):

        alpha, scale = self.alpha(observation)
        beta = self.beta(observation, scale)

        #P(O|lambda)
        P_sequence = np.sum(alpha[-1,:])
        gamma = (alpha*beta)/P_sequence

        return gamma


    def B_update(self, observation):

        new_B = np.zeros((self.state_num,self.obs_num))
        gamma = self.gamma(observation)
        observation_index_list = []
        for i in range(len(observation)-1):
            index = self.observation_dic[obsevation[i]]
            observation_index.append(index)


        for i in range(self.state_num):
            for j in range(self.obs_num):
                obs_occur_index = np.where(observation_index_list = j)[0]
                new_B[i, j] = np.sum(gamma[obs_occur_index,i])/np.sum(gamma[:,i])
        return new_B



    def A_update(self, observation):
        alpha, scale = self.alpha(observation)
        beta = self.beta(observation, scale)
        gamma = self.gamma(obervation)

        sequence_len = len(observation)
        new_A = np.zeros((self.state_num, self.state_num))

        #pseudo matrix: p_it  = b_i(O_t)
        pseudo_matrix = np.zeros((sequence_len, self.state_num))
        for i in range(self.state_num):
            pseudo_matrix[:,i] = [       self.B[i, self.observation_dic[observation[j]]]    for j in range(sequence_len)     ].reshape(-1,1)
            

        P_sequence = np.sum(alpha[-1,:])

        for i in range(self.state_num):
            for j in range(self.state_num):
                di_gamma = alpha[:-1,i].reshape(-1,1) * beta[1:,j].reshape(-1,1)  *   pseudo_matrix[1:, j] * self.A[i,j]* P_sequence
                new_A[i,j] = np.sum(di_gamma)/np.sum(gamma[:-1,i])

        return new_A
                


    def pi_update(self, observation):
        gamma = self.gamma(observation)
        new_pi = gamma[0,:]
        return new_pi
        







        

        
            

    



In [35]:
for i in reversed(range(10-1)):
    print(i)

A = np.zeros((10, 5))
A[9,:] = 1
print(A[-1,:])
B = A[-1,:].reshape(1, -1)
C = A[-1,:].reshape(1, -1)
H = B*C
print('H', H)
print(B)
A = np.concatenate((A, B), axis =0 )
print(A)
# print([i*10 for i in B[i,:]])

# A = ['1', '2','3']
# B ={}
# for index, observation in enumerate(A):
#     B[observation] = index

8
7
6
5
4
3
2
1
0
[1. 1. 1. 1. 1.]
H [[1. 1. 1. 1. 1.]]
[[1. 1. 1. 1. 1.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [14]:
B['1']

0